<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo I: Naive Bayes para Análisis de Sentimientos


Aplicaremos Naive Bayes para el análisis de sentimientos de tweets, lo que haremos en este notebook será:

* Entrenar un modelo de Naive Bayes para el análisis de sentimientos
* Probaremos el modelo creado
* Calcularemos las proporciones de palabras positivas a palabras negativas
* Veremos una función de análisis de errores
* Predeciremos un tweet


In [8]:
import string
from os import getcwd
from utilss import Utilities as prep
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords, twitter_samples



Si aún no se han descargado:
```
nltk.download('stopwords')
nltk.download('twitter_samples')
```

#### añadir una carpeta tmp2, de nuestro workspace local que contenga las dependencias instaladas

In [9]:
filePath = f"{getcwd}/../tmp2/"
nltk.data.path.append(filePath)



In [11]:
# tweets positivos y negativos

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))




# Preprocesamiento de los datos

Para cualquier proyecto de aprendizaje automático, una vez que se haya recopilado los datos, el primer paso es el preprocesamiento.

- **Eliminar el ruido**: eliminar las palabras que no dicen mucho sobre el contenido. Estos incluyen todas las palabras comunes como 'I, you, are, is, etc...' que no nos darían suficiente información sobre el sentimiento.
- También eliminaremos los tickers del mercado de valores, los símbolos de retuits, los hipervínculos y los hashtags porque no pueden brindarle mucha información sobre el sentimiento.
- También hay que eliminar toda la puntuación de un tweet. La razón para hacer esto es porque queremos tratar las palabras con o sin puntuación como la misma palabra, en lugar de tratar las palabras "happy", "happy?", "happy!", "happy," and "happy." como palabras diferentes.
- Por último, desea utilizaremos la técnica de stemming para realizar un seguimiento de una sola variación de cada palabra. En otras palabras, trataremos "motivation", "motivated", y "motivate" de manera similar agrupándolos dentro de la misma raíz de "motiv-".

Recuerden que ya tenemos una función para esto (`process_tweet()`)

In [12]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

obj_prep = prep()

obj_prep.process_tweet(custom_tweet)

This is the Utilities Constructor


['hello', 'great', 'day', ':)', 'good', 'morn']

## Funciones de ayuda

Para entrenar el modelo, necesitaremos construir un diccionario donde las claves sean una tupla (word, label) y los valores sean la frecuencia correspondiente. Tenga en cuenta que las etiquetas que usaremos aquí son 1 para positivo y 0 para negativo.

Implementaremos la función `lookup()` que toma del diccionario `freqs` una palabra y una etiqueta (1 or 0) y retorna el número de veces que la palabra y esa etiqueta aparecen en la collección de tweets.

Por ejemplo: Dado una lista de sentencias en los tweets: `["i am rather excited", "you are rather happy"]` y la etiqueta 1, la función regresara el diccionario que contiene lo siguiente:

{
    ("rather", 1): 2
    ("happi", 1) : 1
    ("excit", 1) : 1
}

- Observe cómo para cada palabra en la cadena dada, se asigna la misma etiqueta 1 a cada palabra.
- Observe cómo las palabras "i" y "am" no se guardan, ya que fue eliminada por process_tweet porque es una stopword.
- Observe cómo la palabra "rather" aparece dos veces en la lista de tweets, por lo que su valor de recuento es 2.

Crearemos una función `count_tweets()` que toma una lista de tweets como entrada, las limpia, y regresa un diccionario.

- La clave en el diccionario es una tupla que contiene la palabra despues del stemming y su etiqueta de clase, Ej. ("happi", 1).
- El valor del key, será la cantidad de veces que esta palabra aparece en la colección de tweets dada (un número entero).


In [19]:
def count_tweets(result, tweets, ys):
    
    for y, tweet in zip(ys,tweets):
        for word in obj_prep.process_tweet(tweet):
            
            pair = (word,y)
            
            if pair in result:
                result[pair] += 1
            
            else:
                result[pair] = 1
                
    return result

            
        
        

In [20]:
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
result={}


In [24]:
f = count_tweets(result, tweets, ys)
f

{('happi', 1): 3, ('trick', 0): 3, ('sad', 0): 3, ('tire', 0): 6}

In [25]:
def lookup(freqs, word, label):
    
    n=0
    pair = (word,label)
    if (pair in freqs):
        n = freqs[pair]
        
    return n



In [27]:
lookup(f, 'tire', 1)

0

## Entrenar el modelo usando Naive Bayes

Naive bayes es un algoritmo que se puede usar para aplicaciones como el análisis de sentimientos. Este algoritmo poco tiempo en entrenarse pero tiene un tiempo de predicción corto.


#### ¿Cómo entrenamos un clasificador con Naive Bayes ?

- La primera parte del entrenamiento de un clasificador Naive Bayes es identificar el número de clases que tiene.
- Crearemos una probabilidad para cada clase.


$P(D_{pos})$ denota la probabilidad que el documento sea positivo.
$P(D_{neg})$ denota la probabilidad que el documento sea negativo.

Utilizaremos las fórmulas de la siguiente manera y almacenaremos los valores en un diccionario:

$$P(D_{pos}) = \frac{D_{pos}}{D}\tag{1}$$

$$P(D_{neg}) = \frac{D_{neg}}{D}\tag{2}$$

Donde $ D $ es el número total de documentos, o tweets en este caso, $ D_ {pos} $ es el número total de tweets positivos y $ D_ {neg} $ es el número total de tweets negativos.

#### Prior y Logprior

The prior probability represents the underlying probability in the target population that a tweet is positive versus negative.  In other words, if we had no specific information and blindly picked a tweet out of the population set, what is the probability that it will be positive versus that it will be negative? That is the "prior".

The prior is the ratio of the probabilities $\frac{P(D_{pos})}{P(D_{neg})}$.
We can take the log of the prior to rescale it, and we'll call this the logprior


La probabilidad previa (prior) representa la probabilidad subyacente de la población (tweets) objetivo de que un tweet sea positivo frente a uno negativo. En otras palabras, si no tuviéramos información específica y seleccionáramos a ciegas un tweet del conjunto de la población, ¿cuál es la probabilidad de que sea positivo frente a que sea negativo? Ese es el "previo".

El "prior" es el ratio de las probabilidades $\frac{P(D_{pos})}{P(D_{neg})}$. Podemos tomar el logaritmo del "prior" para cambiar su escala, y lo llamaremos logprior.


$$\text{logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$$.

Note que $log(\frac{A}{B})$ es igual a $log(A) - log(B)$.  Por lo tanto logprior puede calcularse como la diferencia de dos logaritmos (propiedad de logaritmos).


$$\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg})\tag{3}$$

#### Probabilidad positiva y negativa de una palabra
Para calcular la probabilidad positiva y la probabilidad negativa de una palabra específica en el vocabulario, usaremos las siguientes entradas:

- $freq_{pos}$ y $freq_{neg}$ son las frecuencias de esa palabra específica en la clase positiva o negativa. En otras palabras, la frecuencia positiva de una palabra es el número de veces que se cuenta la palabra con la etiqueta 1.
- $ N_ {pos} $ y $ N_ {neg} $ son el número total de palabras positivas y negativas para todos los documentos (para todos los tweets), respectivamente.

- $V$ es el número de palabras únicas en todo el conjunto de documentos, para todas las clases, ya sean positivas o negativas.

Los usaremos para calcular la probabilidad positiva y negativa de una palabra específica usando la fórmula:

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

Note que añadimos "+1" en el numerador para el smoothing aditivo.  Este [artículo](https://en.wikipedia.org/wiki/Additive_smoothing) explica detalladamente sobre el smoothing aditivo.

#### Log likelihood
Para calcular la loglikelihood (probabilidad mínima) de esa misma palabra, podemos implementar las siguientes ecuaciones:

$$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$$

##### Creación del diccionario de  frecuencias (`freqs` )
- Dada la función `count_tweets ()`, podemos calcular un diccionario llamado `freqs` que contiene todas las frecuencias.
- En este diccionario `freqs`, la clave es la tupla (word,label)
- El valor es el número de veces que ha aparecido.



In [28]:

freqs = count_tweets({},train_x, train_y)
freqs 


{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

### Pipeline para la creación de la función  `train_naive_bayes`
Dado un diccionario de frecuencias, `train_x` (una lista de tweets) y un` train_y` (una lista de etiquetas para cada tweet), implementaremos un clasificador Naive Bayes.

##### Calcular $V$
- Podemos calcular el número de palabras únicas que aparecen en el diccionario `freqs` para obtener $ V $ (usando la función` set`).

##### Calcular $freq_{pos}$ y $freq_{neg}$
- Usando el diccionario `freqs`, se puede calcular la frecuencia positiva y negativa de cada palabra $ freq_ {pos} $ y $ freq_ {neg} $.

##### Calcular $N_{pos}$ y $N_{neg}$
- Usando el diccionario `freqs`, también puede calcular el número total de palabras positivas y el número total de palabras negativas $ N_ {pos} $ y $ N_ {neg} $.

##### Calcular $D$, $D_{pos}$, $D_{neg}$
- Usando la lista de etiquetas de entrada `train_y`, calcularemos el número de documentos (tweets) $ D $, así como el número de documentos positivos (tweets) $ D_ {pos} $ y el número de documentos negativos (tweets) $ D_ { neg} $.
- Calcularemos la probabilidad de que un documento (tweet) sea positivo $ P (D_ {pos}) $, y la probabilidad de que un documento (tweet) sea negativo $ P (D_ {neg}) $

##### Calcularemos el logprior
-  $log(D_{pos}) - log(D_{neg})$

##### Calculate log likelihood
- Finalmente, iteramos sobre cada palabra en el vocabulario, usaremos la función `lookup` para obtener las frecuencias positivas, $ freq_ {pos} $, y las frecuencias negativas, $ freq_ {neg} $, para esa palabra específica.
- Calcularemos la probabilidad positiva de cada palabra $ P (W_ {pos}) $, la probabilidad negativa de cada palabra $ P (W_ {neg}) $ usando las ecuaciones 4 y 5.

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

**Note:** Usaremos un diccionario para almacenar los loglikelihoods de cada palabra. La clave será la palabra, el valor será la probabilidad logarítmica de esa palabra).

- se puede calcular la loglikelihood: $log \left( \frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$.

In [32]:
freqs

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

In [33]:
freqs[('engag', 1.0)]

7

In [30]:
[pair[0] for pair in freqs.keys()]


['followfriday',
 'top',
 'engag',
 'member',
 'commun',
 'week',
 ':)',
 'hey',
 'jame',
 'odd',
 ':/',
 'pleas',
 'call',
 'contact',
 'centr',
 '02392441234',
 'abl',
 'assist',
 'mani',
 'thank',
 'listen',
 'last',
 'night',
 'bleed',
 'amaz',
 'track',
 'scotland',
 'congrat',
 'yeaaah',
 'yipppi',
 'accnt',
 'verifi',
 'rqst',
 'succeed',
 'got',
 'blue',
 'tick',
 'mark',
 'fb',
 'profil',
 '15',
 'day',
 'one',
 'irresist',
 'flipkartfashionfriday',
 'like',
 'keep',
 'love',
 'custom',
 'wait',
 'long',
 'hope',
 'enjoy',
 'happi',
 'friday',
 'lwwf',
 'second',
 'thought',
 '’',
 'enough',
 'time',
 'dd',
 'new',
 'short',
 'enter',
 'system',
 'sheep',
 'must',
 'buy',
 'jgh',
 'go',
 'bayan',
 ':D',
 'bye',
 'act',
 'mischiev',
 'etl',
 'layer',
 'in-hous',
 'wareh',
 'app',
 'katamari',
 'well',
 '…',
 'name',
 'impli',
 ':p',
 'influenc',
 'big',
 '...',
 'juici',
 'selfi',
 'follow',
 'perfect',
 'alreadi',
 'know',
 "what'",
 'great',
 'opportun',
 'junior',
 'triathle

In [31]:
set([pair[0] for pair in freqs.keys()])

{'soundtrack',
 'taknottem',
 'subway',
 'avril',
 'denis',
 'tfw',
 '500',
 'dirtiest',
 'incal',
 'old-porridg',
 'yeaaah',
 'vic',
 'form',
 'christina',
 'somi',
 'setokido',
 'na',
 'presid',
 'pampanga',
 'cunt',
 'carol',
 'nearest',
 'forneret',
 ';)',
 'took',
 'abligaverin',
 '91',
 '2-3',
 'redecor',
 'whsmith',
 '15.90',
 'fela',
 'uncount',
 '😅',
 'today',
 'eyesight',
 'yesss',
 'heart-break',
 'alansmithpart',
 'wknd',
 'cri',
 'soobin',
 '5878e503',
 'parisa',
 'lemon',
 'misspelt',
 'error',
 'kink',
 'post',
 'american',
 'looov',
 '3',
 'alvadhaau',
 'frudg',
 'windowsphon',
 'deathbybaconsmel',
 'ddi',
 'amus',
 'rice',
 '190cr',
 'kyli',
 'onyu',
 '50.000',
 'gana',
 'exam',
 'gun',
 'pic.twitter.com/jxz2lbv6o',
 'lune',
 'sooth',
 'hogan',
 'void',
 'thek',
 'commiss',
 'awhil',
 'relax',
 'incred',
 'ovh',
 '334',
 'iov',
 'jummapray',
 'wo',
 'sign',
 'camsex',
 'aftenoon',
 'lol.hook',
 'all',
 'race',
 'tournament',
 'toast',
 'offroad',
 'empow',
 'msged',
 '

In [37]:
train_y.sum()

4000.0

In [41]:
def trai_naive_bayes(freqs, train_x, train_y):
    
    loglikelihood={}
    logprior = 0
    
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    
    N_pos = 0
    N_neg = 0
    
    for pair in freqs.keys():
        
        if pair[1]>0:
            N_pos += freqs[pair]
        else:
            N_neg += freqs[pair]
    
    D = len(train_y)
    
    D_pos = train_y.sum()
    
    D_neg = D - D_pos
    
    logprior = np.log(D_pos) - np.log(D_neg)
    
    for word in vocab:
        
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)
        
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)
        
        
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
        
    
    return logprior, loglikelihood   
    

In [42]:
logprior, loglikelihood = trai_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))



0.0
9085


In [43]:
loglikelihood

{'soundtrack': 0.6983107032199946,
 'taknottem': -0.687983657899896,
 'subway': -0.687983657899896,
 'avril': 0.6983107032199946,
 'denis': -0.687983657899896,
 'tfw': -0.687983657899896,
 '500': 0.41062863076821354,
 'dirtiest': -0.687983657899896,
 'incal': 0.6983107032199946,
 'old-porridg': 0.6983107032199946,
 'yeaaah': 0.6983107032199946,
 'vic': -0.687983657899896,
 'form': 1.3914578837799398,
 'christina': 0.6983107032199946,
 'somi': 0.6983107032199946,
 'setokido': 0.6983107032199946,
 'na': -1.0934487660080607,
 'presid': 1.1037758113281588,
 'pampanga': 0.6983107032199946,
 'cunt': 0.005163522660049284,
 'carol': 1.1037758113281588,
 'nearest': 0.005163522660049284,
 'forneret': 0.6983107032199946,
 ';)': 3.140657738589199,
 'took': 0.005163522660049284,
 'abligaverin': -1.0934487660080605,
 '91': 1.1037758113281588,
 '2-3': 0.005163522660049284,
 'redecor': 0.6983107032199946,
 'whsmith': 0.6983107032199946,
 '15.90': 0.6983107032199946,
 'fela': -0.687983657899896,
 'unco

# Probando el modelo de Naive Bayes

Ahora que tenemos `logprior` y` loglikelihood`, podemos probar el modelo de Naive Bayes para predecir algunos tweets.

#### Implementación de  `naive_bayes_predict`

Implementaremos la función `naive_bayes_predict` para hacer predicciones de tweets.

* La función toma como entrada el `tweet`,` logprior`, y `loglikelihood`.
* Devuelve la probabilidad de que el tweet pertenezca a la clase positiva o negativa.
* Para cada tweet, se sumarán las probabilidades de cada palabra en el tweet.
* También se agregará el logprior de esta suma para obtener la predicción de sentimiento del tweet

$$ p = logprior + \sum_i^N (loglikelihood_i)$$

#### Nota

Tenga en cuenta que calculamos el "prior" a partir de los datos de entrenamiento y que los datos de entrenamiento se dividen uniformemente entre etiquetas positivas y negativas (4000 tweets positivos y 4000 negativos). Esto significa que el ratio de positivo a negativo 1, y el logprior es 0.

El valor de 0.0 significa que cuando agregamos el logprior al likelihood, simplemente estamos agregando un valor de cero a la probabilidad logarítmica. Sin embargo, recuerde incluir el logprior, porque siempre que los datos no estén perfectamente equilibrados, el logprior será un valor distinto de cero.


In [44]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    
    word_l = obj_prep.process_tweet(tweet)
    
    p=0
    
    for word in word_l:
        
        if word in loglikelihood:
            p += loglikelihood[word]
            
    
    return p
    

In [48]:
my_tweet_ernesto = 'Folks the Delta variant is no joke. Please protect yourself and your loved ones – get vaccinated.'
my_tweet_guille = 'Gary Oldman sucks and his accents are fucking annoying as shit'
my_tweet_ed = "It wasn't held at the 'Trump' Turnberry. Back then en it was a classy and prestigious joint, well maintained and considered a home for gentleman"


In [51]:
p = naive_bayes_predict(my_tweet_guille, logprior, loglikelihood)
print(p)

-3.552537802495815


#### Implementando  test_naive_bayes

* Implementaremos `test_naive_bayes` la función para ver el accuracy de las predicciones. 
* La función toma de `test_x`, `test_y`,el log_prior, y el loglikelihood
* Regresa el ccuracy del modelo
* Primeramente, usaremos la función `naive_bayes_predict` para hacer las predicciones para cada tweet en text_x.

In [4]:
tweet_sample= ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']


### Predecir un tweet propio

# Filtrado de palabras por ratios positivos y negativos

- Algunas palabras tienen un conteo de etiquetas más positivos que otras y pueden considerarse "más positivas". Asimismo, algunas palabras pueden considerarse más negativas que otras.

- Una forma de definir el nivel de positividad o negatividad, sin calcular la probabilidad logarítmica, es comparar la frecuencia positiva con la negativa de la palabra.

    - Tenga en cuenta que también podemos utilizar los cálculos de probabilidad logarítmica para comparar la positividad o negatividad relativa de las palabras.
    
- Podemos calcular el ratio de las frecuencias positivas y negativas de una palabra.
- Una vez que podamos calcular estos ratios, también podemos filtrar un subconjunto de palabras que tengan una proporción mínima de positividad / negatividad o superior.
- De manera similar, también podemos filtrar un subconjunto de palabras que tienen una proporción máxima de positividad / negatividad o menor (palabras que son menos negativas, o incluso más negativas que un umbral dado).

#### Implementación `get_ratio()`

- Dado el diccionario de palabras `freqs` y una palabra en particular, usaremos` lookup (freqs, word, 1) `para obtener el recuento positivo de la palabra.
- De manera similar, usaremos la función `lookup ()` para obtener el recuento negativo de esa palabra.
- Calcularemos la proporción de positivo dividido por conteos negativos.

$$ ratio = \frac{\text{pos_words} + 1}{\text{neg_words} + 1} $$

Where pos_words and neg_words correspond to the frequency of the words in their respective classes. 
<table>
    <tr>
        <td>
            <b>Words</b>
        </td>
        <td>
        Positive word count
        </td>
         <td>
        Negative Word Count
        </td>
  </tr>
    <tr>
        <td>
        glad
        </td>
         <td>
        41
        </td>
    <td>
        2
        </td>
  </tr>
    <tr>
        <td>
        arriv
        </td>
         <td>
        57
        </td>
    <td>
        4
        </td>
  </tr>
    <tr>
        <td>
        :(
        </td>
         <td>
        1
        </td>
    <td>
        3663
        </td>
  </tr>
    <tr>
        <td>
        :-(
        </td>
         <td>
        0
        </td>
    <td>
        378
        </td>
  </tr>
</table>

#### Implementación `get_words_by_threshold(freqs,label,threshold)`

* Si establecemos label en 1, buscaremos todas las palabras cuyo umbral de positivo / negativo sea mayoy o igual al umbral.
* Si establecemos la etiqueta en 0, buscaremos todas las palabras cuyo umbral de positivo / negativo sea menor o igual al umbral.
* Utilizaremos la función `get_ratio ()` para obtener un diccionario que contenga el recuento positivo, el recuento negativo y la proporción de recuentos positivos y negativos.
* Agregaremos un diccionario a una lista, donde la clave es la palabra y el diccionario es el diccionario `pos_neg_ratio` que es devuelto por la función` get_ratio () `.

Un ejemplo de este diccionario sería:
```
{'happi':
    {'positive': 10, 'negative': 20, 'ratio': 0.5}
}
```

**Observe la diferencia entre los ratios positivos y negativos. Emojis como :( y palabras como 'me' tienden a tener una connotación negativa. Otras palabras como 'glad', 'comunity' y 'arrives' tienden a encontrarse en los tw-eets positivos.**


# Análisis de Error

En esta parte, veremos algunos tweets que el modelo no clasificó correctamente. ¿Por qué crees que ocurrieron las clasificaciones erróneas? ¿Hubo alguna suposición hecha por el modelo de Naive Bayes?

# Predice con tu propio tweet


In [6]:
# 
my_tweet = 'I am happy because I am in the best course of NLP :) #amazing'



Congratulations on completing this assignment. See you next week!